In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import rfp_utils
import utils

In [ ]:
# FRF teórica, con 2 picos acoplados y un 3ero lejano
f_n = np.array(([10, 12, 60]))
w_n = 2*np.pi*f_n
xi_n = np.array(([0.05, 0.045, 0.025]))
r_n = np.array(([200*np.exp(1j*np.pi), 150*np.exp(1j*np.pi), 2500]))

w = np.linspace(0,400,4001)
f = w/2/np.pi

In [ ]:
frf = np.zeros(len(w), dtype=complex)
for k in range(len(r_n)):
    frf += r_n[k] / (w_n[k]**2 - w**2 + 1j*2*w*w_n[k]*xi_n[k])

In [ ]:
plt.plot(f, abs(frf))
plt.show()

In [ ]:
# Parametrización modal del 1er pico por RFP
modal_params, alpha = rfp_utils.rfp(frf[400:700], w[400:700], 3)
freq_n, xi_n, modal_mag_n, modal_ang_n = modal_params

print('Frequencias: {} Hz'.format(np.round(freq_n, 2)))
print('Amortiguamientos: {}'.format(np.round(xi_n, 3)))
plt.semilogy(f, abs(frf))
plt.plot(f[400:700], abs(alpha))
plt.show()

### GRFP

In [ ]:
# Ampliación de los residuos a varias mediciones
r_n0 = np.array(([75*np.exp(1j*np.pi), 75*np.exp(0), 2500*np.exp(0)]))
r_n1 = np.array(([150*np.exp(1j*np.pi), 150*np.exp(1j*np.pi), 0]))
r_n2 = np.array(([200*np.exp(1j*np.pi), 75*np.exp(0), 2500*np.exp(1j*np.pi)]))

r_n = np.array([r_n0, r_n1, r_n2])
n_dof = r_n.shape[0]
n_modes = r_n.shape[1]
utils.plot_column_modes(r_n)

In [ ]:
# Obtención de las distintas FRF
frf = np.zeros((len(w), n_dof), dtype=complex)
for dof in range(n_dof):
    for mode in range(n_modes):
        frf[:, dof] += r_n[dof, mode] / (w_n[mode]**2 - w**2 + 1j*2*w*w_n[mode]*xi_n[mode])

plt.semilogy(f, np.abs(frf))
plt.show()

### Parametrización por GRFP de los 1eros 2 picos

In [ ]:
# Obtencion del polinomio denominador
idx12 = np.linspace(300, 1000, 701, dtype=int)
w12 = w[idx12]
denominator, denominator_coeff = rfp_utils.grfp_denominator(frf[idx12, :], w12, 3)

plt.semilogy(w12, np.abs(1/denominator))
plt.show()

In [ ]:
# Obtencion de los parametros modales de cada FRF
frf_estimates, modal_params = rfp_utils.grfp_parameters(frf[idx12, :], w12, denominator, denominator_coeff, 3)

In [ ]:
freq_n, xi_n, modal_mag_n, modal_ang_n = modal_params
mode_vectors = (modal_mag_n*np.exp(1j*modal_ang_n)).T

print('Frequencias: {} Hz'.format(np.round(freq_n, 2)))
print('Amortiguamientos: {}'.format(np.round(xi_n, 3)))
utils.plot_column_modes(mode_vectors)

In [ ]:
fig, ax= plt.subplots(1, n_dof, figsize=(17,6))
[ax.flat[i].grid(True, markevery=1) for i in range(len(ax.flat))]

for dof in range(n_dof):
    ax.flat[dof].semilogy(w12, np.abs(frf[idx12, dof]), label='teorica')
    ax.flat[dof].semilogy(w12, np.abs(frf_estimates[:, dof]), label='estimada')
    ax.flat[dof].legend()
    plt.plot()